In [11]:
import os

folderList = os.listdir("/home/jupyter/highway")
print(len(folderList), folderList)

2 ['val', 'train']


In [12]:
!pip install lxml

In [13]:
import numpy as np
import lxml
import os

from lxml import etree

CLASSES = ["car", "bus", "truck"]

def to_yolov8(y):
  """
  # change to yolo v8 format
  # [x_top_left, y_top_left, x_bottom_right, y_bottom_right] to
  # [x_center, y_center, width, height]
  """
  width = y[2] - y[0]
  height = y[3] - y[1]

  if width < 0 or height < 0:
      print("ERROR: negative width or height ", width, height, y)
      raise AssertionError("Negative width or height")
  return (y[0] + (width/2)), (y[1] + (height/2)), width, height


def load_xml_annotations(f):
  tree = etree.parse(f)
  anns = []
  for dim in tree.xpath("image"):
    image_filename = dim.attrib["name"]
    width = int(dim.attrib["width"])
    height = int(dim.attrib["height"])
    # print(image_filename)
    # print(len(dim.xpath("box")))
    boxes = []
    for box in dim.xpath("box"):
      label = CLASSES.index(box.attrib["label"])
      xtl, ytl = box.attrib["xtl"], box.attrib["ytl"]
      xbr, ybr = box.attrib["xbr"], box.attrib["ybr"]

      xc, yc, w, h = to_yolov8([float(xtl), float(ytl), float(xbr), float(ybr)])
      boxes.append([label, round(xc/width, 5), round(yc/height, 5), round(w/width, 5), round(h/height, 5)])

    anns.append([image_filename, width, height, boxes])

  return np.array(anns, dtype="object")


def write_yolov8_txt(folder, annotation):
  out_filename = os.path.join(folder,annotation[0][:-3])
  out_filename = os.path.splitext(out_filename)[0]
  out_filename = out_filename+'.txt'
  #print(out_filename)
  f = open(out_filename,"w+")
  for box in annotation[3]:
    f.write("{} {} {} {} {}\n".format(box[0], box[1], box[2], box[3], box[4]))
  f.close()

## xml 파일 하나 변환하기

In [34]:
!mkdir /home/jupyter/highway/train/txt_total

In [35]:
%%time
dataPath='/home/jupyter/highway/train/labels_total/'
destPath='/home/jupyter/highway/train/txt_total'

#basePath = os.getcwd()

# 파일명을 설정한다.
xmlFile='Suwon_CH01_20200720_1700_MON_9m_NH_highway_TW5_sunny_FHD.xml'

# 현재 경로 + 파일명
label_file = os.path.join(dataPath, xmlFile)

# XML을 TXT로 변환한다.
anns = load_xml_annotations(label_file)
print(len(anns))

## 내 파일 형식은 labels 폴더 아래에 txt 파일을 전부 푸는 것이기 때문에 아래 2개코드는 주석처리하였습니다.
## lables/폴더명/txt파일 형식으로 지정하면 아래 코드 사용
folderName=os.path.splitext(xmlFile)[0]
os.makedirs(os.path.join(destPath,folderName), exist_ok=True)


# 이미지 파일별로 TXT파일을 저장한다.
for ann in anns:
  write_yolov8_txt(os.path.join(destPath,folderName), ann)

150
CPU times: user 55.2 ms, sys: 2.15 ms, total: 57.4 ms
Wall time: 55.4 ms


In [36]:
!rm -rf highway/train/labels/Suwon_CH01_20200720_1700_MON_9m_NH_highway_TW5_sunny_FHD

## train : labels_total폴더안에 있는 모든 xml파일을 변환하기

In [37]:
# xml 목록 읽어오기
from glob import glob
dataPath='/home/jupyter/highway/train/labels_total/'
xmlList = glob(dataPath+'*.xml')
basename=[]
for xmlFile in xmlList:
    basename.append(os.path.basename(xmlFile))

In [42]:
#basename

In [45]:
%%time
dataPath='/home/jupyter/highway/train/labels_total/'
destPath='/home/jupyter/highway/train/txt_total'

for xmlFile in basename:
    # 현재 경로 + 파일명
    label_file = os.path.join(dataPath, xmlFile)

    # XML을 TXT로 변환한다.
    anns = load_xml_annotations(label_file)
    #print(len(anns))
    folderName=os.path.splitext(xmlFile)[0]
    print(folderName)
    os.makedirs(os.path.join(destPath,folderName), exist_ok=True)


    # 이미지 파일별로 TXT파일을 저장한다.
    for ann in anns:
      write_yolov8_txt(os.path.join(destPath,folderName), ann)

  3%|▎         | 5/168 [00:00<00:07, 20.75it/s]

Suwon_CH04_20200721_1700_TUE_9m_RH_highway_OW5_sunny_FHD
Suwon_CH03_20200720_1700_MON_9m_NH_highway_OW5_sunny_FHD
Suwon_CH02_20201012_1928_MON_9m_RH_highway_TW5_sunny_FHD
Suwon_CH04_20200722_1900_WED_9m_NH_highway_OW5_sunny_FHD
Suwon_CH03_20200721_1530_TUE_9m_NH_highway_OW5_sunny_FHD
Suwon_CH02_20200721_2100_TUE_9m_NH_highway_TW5_sunny_FHD


  5%|▍         | 8/168 [00:00<00:08, 17.87it/s]

Suwon_CH02_20200722_1700_WED_9m_RH_highway_TW5_sunny_FHD
Suwon_CH01_20201012_1823_MON_9m_RH_highway_TW5_sunny_FHD
Suwon_CH04_20200722_1630_WED_9m_NH_highway_OW5_rainy_FHD
Suwon_CH01_20200722_1730_WED_9m_RH_highway_TW5_sunny_FHD


  8%|▊         | 14/168 [00:00<00:06, 22.67it/s]

...

Suwon_CH02_20201010_1908_SAT_9m_NH_highway_TW5_sunny_FHD
Suwon_CH04_20201011_1748_SUN_9m_RH_highway_OW5_sunny_FHD
Suwon_CH04_20200722_1730_WED_9m_NH_highway_OW5_sunny_FHD
Suwon_CH04_20201010_1748_SAT_9m_RH_highway_OW5_sunny_FHD
Suwon_CH03_20200722_1830_WED_9m_RH_highway_OW5_sunny_FHD
Suwon_CH01_20201011_1830_SUN_9m_RH_highway_TW5_sunny_FHD
Suwon_CH01_20200722_1900_WED_9m_RH_highway_TW5_sunny_FHD


 98%|█████████▊| 165/168 [00:07<00:00, 25.04it/s]

Suwon_CH01_20200721_1630_TUE_9m_NH_highway_TW5_sunny_FHD
Suwon_CH04_20201011_1717_SUN_9m_NH_highway_OW5_sunny_FHD
Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD
Suwon_CH01_20200721_1530_TUE_9m_NH_highway_TW5_sunny_FHD
Suwon_CH04_20200720_1730_MON_9m_NH_highway_OW5_sunny_FHD


100%|██████████| 168/168 [00:07<00:00, 23.03it/s]

Suwon_CH03_20201010_1913_SAT_9m_NH_highway_OW5_sunny_FHD
Suwon_CH02_20200721_1600_TUE_9m_NH_highway_TW5_sunny_FHD
CPU times: user 5.68 s, sys: 780 ms, total: 6.46 s
Wall time: 7.3 s


In [49]:
len(os.listdir('/home/jupyter/highway/train/txt_total'))

168

## val : labels_total폴더안에 있는 모든 xml파일을 변환하기

In [67]:
#!mkdir /home/jupyter/highway/val/txt_total

In [68]:
%%time
# xml 목록 읽어오기
from glob import glob
dataPath='/home/jupyter/highway/val/labels_total/'
xmlList = glob(dataPath+'*.xml')
basename=[]
for xmlFile in xmlList:
    basename.append(os.path.basename(xmlFile))
    
dataPath='/home/jupyter/highway/val/labels_total'
destPath='/home/jupyter/highway/val/txt_total'

for xmlFile in basename:
    # 현재 경로 + 파일명
    label_file = os.path.join(dataPath, xmlFile)

    # XML을 TXT로 변환한다.
    anns = load_xml_annotations(label_file)
    #print(len(anns))
    folderName=os.path.splitext(xmlFile)[0]
    print(folderName)
    os.makedirs(os.path.join(destPath,folderName), exist_ok=True)


    # 이미지 파일별로 TXT파일을 저장한다.
    for ann in anns:
      write_yolov8_txt(os.path.join(destPath,folderName), ann)

Suwon_CH04_20201012_1838_MON_9m_RH_highway_OW5_sunny_FHD
Suwon_CH02_20200721_2030_TUE_9m_NH_highway_TW5_sunny_FHD
Suwon_CH03_20201012_1933_MON_9m_RH_highway_OW5_sunny_FHD
Suwon_CH02_20201213_0933_SUN_9m_NH_highway_TW5_snow_FHD
Suwon_CH02_20201011_1806_SUN_9m_RH_highway_TW5_sunny_FHD
Suwon_CH02_20200722_1730_WED_9m_NH_highway_TW5_sunny_FHD
Suwon_CH01_20200722_1930_WED_9m_NH_highway_TW5_sunny_FHD
Suwon_CH04_20201010_1818_SAT_9m_RH_highway_OW5_sunny_FHD
Suwon_CH02_20200720_2130_MON_9m_NH_highway_TW5_sunny_FHD
Suwon_CH03_20200720_2030_MON_9m_NH_highway_OW5_sunny_FHD
Suwon_CH04_20200721_1830_TUE_9m_RH_highway_OW5_sunny_FHD
Suwon_CH03_20201011_1742_SUN_9m_RH_highway_OW5_sunny_FHD
Suwon_CH01_20201213_1200_SUN_9m_NH_highway_TW5_snow_FHD
Suwon_CH04_20200722_1600_WED_9m_NH_highway_OW5_rainy_FHD
Suwon_CH03_20200722_1700_WED_9m_NH_highway_OW5_sunny_FHD
Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD
Suwon_CH03_20200721_2000_TUE_9m_NH_highway_OW5_sunny_FHD
Suwon_CH04_20200720_1830_MON_9m_N

In [69]:
import glob
a = glob.glob("/home/jupyter/highway/train/txt_total/*/*")
b = glob.glob("/home/jupyter/highway/val/txt_total/*/*")
len(a), len(b)

(0, 3334)